## Satisfiability

In this section, we study the DPLL algorithm (for Davis-Putnam-Logemann-Loveland) for solving satisfiability problems. In particular, we present the version with deriving conflict clauses and non-chronological backtracking. If the formula is unsatisfiable, the algorithms returns a certificate of unsatisfiability as a series of resolution steps.

The material in this section is based on Chapter 3 and 4 from Handbook of Satisfiability.

The DPLL algorithm works with a formula in CNF form, for example:

$$ (\neg x_1 \vee x_2) \wedge (\neg x_2 \vee \neg x_3 \vee x_4) \wedge (\neg x_1 \vee \neg x_3 \vee \neg x_4) $$

This formula can be presented as a list of lists of literals, where each literal is either $P$ (represented as the pair $(P, \mathit{True})$) or $\neg P$ (represented as the pair $(P, \mathit{False})$). For example, the above formula is represented as:

In [1]:
cnf = [
    [('x1', False), ('x2', True)],
    [('x2', False), ('x3', False), ('x4', True)],
    [('x1', False), ('x3', False), ('x4', False)]
]

The printing function for CNF's can be written as follows:

In [2]:
def str_of_literal(lit):
    return lit[0] if lit[1] else '¬' + lit[0]

def str_of_clause(clause):
    if len(clause) == 0:
        return 'False'
    else:
        return ' ∨ '.join(str_of_literal(lit) for lit in clause)

def str_of_cnf(cnf):
    if len(cnf) == 0:
        return 'True'
    else:
        return ' ∧ '.join('(' + str_of_clause(clause) + ')' for clause in cnf)

In [3]:
print(str_of_cnf(cnf))

(¬x1 ∨ x2) ∧ (¬x2 ∨ ¬x3 ∨ x4) ∧ (¬x1 ∨ ¬x3 ∨ ¬x4)


The basic DPLL algorithm is quite simple to describe: if the current CNF allows us to immediately derive the value of any literals, do so. Otherwise, pick a literal that has not been assigned a value, assign either $\mathit{True}$ or $\mathit{False}$ to the literal, then try to derive the values of other literals as far as possible.

We maintain a dictionary of assignment of variables. To each variable, we associate a tuple $(\mathit{val}, \mathit{is\_decide}, \mathit{level}, \mathit{clause})$. The meaning of each component is as follows:

* $\mathit{val}$ is the assigned value of the variable.
* $\mathit{is\_decide} = \mathit{True}$ if the assignment is a decision. Otherwise the assignment is derived by unit propagation.
* $\mathit{level}$ gives the number of decided assignments at or before this assignment. The level is 0 for the initial assigments due to unit propagation, 1 for the first decision and unit propagation resulting from it, etc.
* $\mathit{clause}$ gives, for each derived assignment, the index of the clause used to derived it. This is useful for reconstructing the proof later. For decision assignments this is $\mathit{None}$.

In [4]:
# Mapping from assigned variables to its information.
assigns = dict()

# Current level
level = 0

We first define the unit propagation function. This function goes over the clauses. For each clause that is not satisfied by the assignment, and that has only one literal remaining, it assigns the appropriate value to the literal so the clause can be satisfied. The function stops either when there is no propagation remaining (in which case it returns `None`), or if it discovered a conflict clause (in which case it returns (`conflict`, $id$), where $id$ is the conflict clause, or if a satisfying assignment is discovered (in which case it returns `satisfiable`).

In [5]:
def unit_propagate():
    global assigns, level
    while True:
        has_unsatisfied = False  # whether there is still clause to be satisfied
        has_propagate = False  # whether a propagation has occurred
        for clause_id, clause in enumerate(cnf):
            satisfied = False  # whether the current clause is satisfied
            unassigned = []  # list of unassigned literals
            
            # Iterate over the literals, check if the existing assignment satisfies
            # the clause, and if not, what are the unassigned literals.
            for lit in clause:
                name, val = lit
                if name in assigns:
                    if val == assigns[name][0]:
                        satisfied = True
                        break
                else:
                    unassigned.append(lit)
                    
            # If the clause is not already satisfied, no unassigned literals implies
            # conflict. One unassigned literals implies possibility for unit propagation.
            if not satisfied:
                if len(unassigned) == 0:
                    return 'conflict', clause_id
                elif len(unassigned) == 1:
                    name, val = unassigned[0]
                    print('Unit propagate %s = %s using clause %s' % (name, val, clause_id))
                    assigns[name] = (val, False, level, clause_id)
                    has_propagate = True
                    break
                else:
                    has_unsatisfied = True
                    
        if not has_propagate:
            if not has_unsatisfied:
                return 'satisfiable'
            else:
                return None

We test this function on the initial state:

In [6]:
level = 1
assigns = {}
print(unit_propagate())

None


As expected, nothing is done because no unit propagation is available at this step. Next, let's guess $x_1 = \mathit{True}$, and start unit propagation from there:

In [7]:
level = 1
assigns = {'x1': (True, True, 1, None)}
print(unit_propagate())
print(assigns)

Unit propagate x2 = True using clause 0
None
{'x1': (True, True, 1, None), 'x2': (True, False, 1, 0)}


We see that the algorithm is able to propagate $x_2 = \mathit{True}$, but no more. If we further guess $x_3 = \mathit{True}$, it will propagate the value of $x_4$, and then derive a contradiction (from clause with $id = 2$).

In [8]:
level = 2
assigns = {'x1': (True, True, 1, None), 'x2': (True, False, 1, 0), 'x3': (True, True, 2, None)}
print(unit_propagate())

Unit propagate x4 = True using clause 1
('conflict', 2)


If a conflict occurred, we can analyze the conflict to see which decision variables it depends on. First, we define a function computing the resolution of two clauses on a literal. This function assumes (does not check) that the inputs are valid.

In [9]:
def resolution(clause1, clause2, name):
    lit1 = [lit for lit in clause1 if lit[0] != name]
    lit2 = [lit for lit in clause2 if lit[0] != name]
    return list(set(lit1 + lit2))

We can briefly test this function:

In [10]:
print(resolution([('x1', True), ('x2', False)], [('x1', False), ('x3', True)], 'x1'))
print(resolution([('x1', False), ('x2', False)], [('x1', True), ('x2', False)], 'x1'))

[('x2', False), ('x3', True)]
[('x2', False)]


In [11]:
def analyze_conflict(clause_id):
    clause = cnf[clause_id]
    proof = [clause_id]
    print('Analyze conflict on clause %s: %s' % (clause_id, str_of_clause(clause)))
    while True:
        has_resolution = False
        for lit in clause:
            name, val = lit
            assert name in assigns and val != assigns[name][0]
            _, is_decide, level, propagate_id = assigns[name]
            if not is_decide:
                has_resolution = True
                proof.append(propagate_id)
                clause = resolution(clause, cnf[propagate_id], name)
                print('Resolution with clause %s on atom %s, obtaining %s' % (propagate_id, name, str_of_clause(clause)))
                break

        if not has_resolution:
            break

    return proof, clause

The function returns a list of resolution steps and the resulting clause.

In [12]:
print(analyze_conflict(2))

Analyze conflict on clause 2: ¬x1 ∨ ¬x3 ∨ ¬x4
Resolution with clause 1 on atom x4, obtaining ¬x3 ∨ ¬x1 ∨ ¬x2
Resolution with clause 0 on atom x2, obtaining ¬x3 ∨ ¬x1
([2, 1, 0], [('x3', False), ('x1', False)])


We now implement the backtracking function. When a conflict occurs, it is analyzed and the list of assignments is backtracked according the analysis. The analysis returns the list of decision assignments the conflict is based on. Sort the decision assignments by level, and suppose the last two levels are $l_2$ and $l_1$. We backtrack to the level $l_2$, that is removing all decisions at and after level $l_2+1$. If there is only one decision assignment in the conflict, we backtrack to level $0$, that is removing all decisions at and after level $1$. We also append the conflict clause (the *learned* clause) to the list of clauses, and remember its resolution proof. The conflict clause allows at least one unit propagation, assigning the variable originally decided at level $l_1$. If there is no decision assignments in the conflict, it means we have proved the empty clause, and the overall formula is unsatisfiable.

In [13]:
# Records proofs of learned clauses
proofs = dict()

def backtrack(clause_id):
    # Analyze conflict, record the new clause and its proof
    proof, clause = analyze_conflict(clause_id)
    new_id = len(cnf)
    cnf.append(clause)
    proofs[new_id] = proof

    # Sort the clause by level, find the second to last level
    if len(clause) == 0:
        return 'unsatisfiable'
    elif len(clause) == 1:
        backtrack_level = 0
    else:
        clause = sorted(clause, key=lambda lit: assigns[lit[0]][2])
        name, _ = clause[-2]
        backtrack_level = assigns[name][2]
        
    # Backtrack to that level
    assigned_names = list(assigns.keys())
    for name in assigned_names:
        if assigns[name][2] > backtrack_level:
            del assigns[name]
            
    return backtrack_level

We test the backtrack function from the current state of assignments:

In [14]:
assigns = {'x1': (True, True, 1, None), 'x2': (True, False, 1, 0), 'x3': (True, True, 2, None), 'x4': (True, False, 2, 1)}
cnf = [
    [('x1', False), ('x2', True)],
    [('x2', False), ('x3', False), ('x4', True)],
    [('x1', False), ('x3', False), ('x4', False)]
]
proofs = dict()

print('new level:', backtrack(2))
print(assigns)
print(str_of_cnf(cnf))
print(proofs)

Analyze conflict on clause 2: ¬x1 ∨ ¬x3 ∨ ¬x4
Resolution with clause 1 on atom x4, obtaining ¬x3 ∨ ¬x1 ∨ ¬x2
Resolution with clause 0 on atom x2, obtaining ¬x3 ∨ ¬x1
new level: 1
{'x1': (True, True, 1, None), 'x2': (True, False, 1, 0)}
(¬x1 ∨ x2) ∧ (¬x2 ∨ ¬x3 ∨ x4) ∧ (¬x1 ∨ ¬x3 ∨ ¬x4) ∧ (¬x3 ∨ ¬x1)
{3: [2, 1, 0]}


Now, we put all these together in a full algorithm. First, unit propagation is performed to find derived assignments at level 0. Then the main loop begins. At each iteration of the loop, we first decide on a variable that is not yet assigned. Then perform unit propagation. If unit propagation results in satisfiable, we return the result. If it stops without finding a conflict, we start another iteration. If it finds a conflict, we perform backtracking.

In [15]:
def dpll():
    global level, assigns, proofs
    level = 0
    assigns = dict()
    proofs = dict()
    
    # Find set of all variables
    variables = set()
    for clause in cnf:
        for name, _ in clause:
            variables.add(name)

    propagate_res = unit_propagate()
    while True:
        if propagate_res == 'satisfiable':
            # Formula is satisfiable, return only assigned values
            assignment = dict((name, val) for name, (val, _, _, _) in assigns.items())
            return 'satisfiable', assignment
        elif propagate_res is None:
            # Unit propagation stops, choose a new variable
            level += 1
            for var in variables:
                if var not in assigns:
                    assigns[var] = (True, True, level, None)
                    break
        else:
            # Conflict occurs, backtrack
            _, clause_id = propagate_res
            backtrack_res = backtrack(clause_id)
            if backtrack_res == 'unsatisfiable':
                # Formula is unsatisfiable, return the proof
                return 'unsatisfiable', proofs
            else:
                level = backtrack_res
            
        propagate_res = unit_propagate()

In [16]:
cnf = [
    [('x1', False), ('x2', True)],
    [('x2', False), ('x3', False), ('x4', True)],
    [('x1', False), ('x3', False), ('x4', False)]
]
print(str_of_cnf(cnf))
print(dpll())

(¬x1 ∨ x2) ∧ (¬x2 ∨ ¬x3 ∨ x4) ∧ (¬x1 ∨ ¬x3 ∨ ¬x4)
Unit propagate x2 = True using clause 0
Unit propagate x3 = False using clause 2
('satisfiable', {'x1': True, 'x2': True, 'x4': True, 'x3': False})


The result indicates that the formula is satisfiable by assignment $x_1$ and $x_2$ to true, and $x_3$ to false. Let's test on an unsatisfiable formula:

In [17]:
cnf = [
    [('x1', True), ('x2', True)],
    [('x1', True), ('x2', False)],
    [('x1', False), ('x2', True)],
    [('x1', False), ('x2', False)]
]
print(str_of_cnf(cnf))
print(dpll())

(x1 ∨ x2) ∧ (x1 ∨ ¬x2) ∧ (¬x1 ∨ x2) ∧ (¬x1 ∨ ¬x2)
Unit propagate x2 = True using clause 2
Analyze conflict on clause 3: ¬x1 ∨ ¬x2
Resolution with clause 2 on atom x2, obtaining ¬x1
Unit propagate x1 = False using clause 4
Unit propagate x2 = True using clause 0
Analyze conflict on clause 1: x1 ∨ ¬x2
Resolution with clause 4 on atom x1, obtaining ¬x2
Resolution with clause 0 on atom x2, obtaining x1
Resolution with clause 4 on atom x1, obtaining False
('unsatisfiable', {4: [3, 2], 5: [1, 4, 0, 4]})


The result indicates that the formula is unsatisfiable, returning a proof based on resolution. For good measure, let's also test the code on some corner cases:

In [18]:
cnf = []  # trivially true
print(str_of_cnf(cnf))
print(dpll())

True
('satisfiable', {})


In [19]:
cnf = [[]]  # trivially false
print(str_of_cnf(cnf))
print(dpll())

(False)
Analyze conflict on clause 0: False
('unsatisfiable', {1: [0]})


The full code for DPLL can be wrapped up in a function:

In [20]:
from copy import copy

def dpll(cnf, *, debug=False):
    cnf = copy(cnf)  # avoid modifying the input
    assigns = dict()
    level = 0
    proofs = dict()
    
    def print_debug(s):
        if debug:
            print(s)

    def unit_propagate():
        while True:
            has_unsatisfied = False  # whether there is still clause to be satisfied
            has_propagate = False  # whether a propagation has occurred
            for clause_id, clause in enumerate(cnf):
                satisfied = False  # whether the current clause is satisfied
                unassigned = []  # list of unassigned literals

                # Iterate over the literals, check if the existing assignment satisfies
                # the clause, and if not, what are the unassigned literals.
                for lit in clause:
                    name, val = lit
                    if name in assigns:
                        if val == assigns[name][0]:
                            satisfied = True
                            break
                    else:
                        unassigned.append(lit)

                # If the clause is not already satisfied, no unassigned literals implies
                # conflict. One unassigned literals implies possibility for unit propagation.
                if not satisfied:
                    if len(unassigned) == 0:
                        return 'conflict', clause_id
                    elif len(unassigned) == 1:
                        name, val = unassigned[0]
                        print_debug('Unit propagate %s = %s using clause %s' % (name, val, clause_id))
                        assigns[name] = (val, False, level, clause_id)
                        has_propagate = True
                        break
                    else:
                        has_unsatisfied = True

            if not has_propagate:
                if not has_unsatisfied:
                    return 'satisfiable'
                else:
                    return None

    def analyze_conflict(clause_id):
        clause = cnf[clause_id]
        proof = [clause_id]
        print_debug('Analyze conflict on clause %s: %s' % (clause_id, str_of_clause(clause)))
        while True:
            has_resolution = False
            for lit in clause:
                name, val = lit
                assert name in assigns and val != assigns[name][0]
                _, is_decide, level, propagate_id = assigns[name]
                if not is_decide:
                    has_resolution = True
                    proof.append(propagate_id)
                    clause = resolution(clause, cnf[propagate_id], name)
                    print_debug('Resolution with clause %s on atom %s, obtaining %s' % (propagate_id, name, str_of_clause(clause)))
                    break

            if not has_resolution:
                break

        return proof, clause
    
    
    def backtrack(clause_id):
        # Analyze conflict, record the new clause and its proof
        proof, clause = analyze_conflict(clause_id)
        new_id = len(cnf)
        cnf.append(clause)
        proofs[new_id] = proof

        # Sort the clause by level, find the second to last level
        if len(clause) == 0:
            return 'unsatisfiable'
        elif len(clause) == 1:
            backtrack_level = 0
        else:
            clause = sorted(clause, key=lambda lit: assigns[lit[0]][2])
            name, _ = clause[-2]
            backtrack_level = assigns[name][2]

        # Backtrack to that level
        assigned_names = list(assigns.keys())
        for name in assigned_names:
            if assigns[name][2] > backtrack_level:
                del assigns[name]

        return backtrack_level
    
    # Find set of all variables
    variables = set()
    for clause in cnf:
        for name, _ in clause:
            variables.add(name)

    propagate_res = unit_propagate()
    while True:
        if propagate_res == 'satisfiable':
            # Formula is satisfiable, return only assigned values
            assignment = dict((name, val) for name, (val, _, _, _) in assigns.items())
            return 'satisfiable', assignment
        elif propagate_res is None:
            # Unit propagation stops, choose a new variable
            level += 1
            for var in variables:
                if var not in assigns:
                    assigns[var] = (True, True, level, None)
                    break
        else:
            # Conflict occurs, backtrack
            _, clause_id = propagate_res
            backtrack_res = backtrack(clause_id)
            if backtrack_res == 'unsatisfiable':
                # Formula is unsatisfiable, return the proof
                return 'unsatisfiable', proofs
            else:
                level = backtrack_res
            
        propagate_res = unit_propagate()

In [21]:
print(dpll([
    [('x1', False), ('x2', True)],
    [('x2', False), ('x3', False), ('x4', True)],
    [('x1', False), ('x3', False), ('x4', False)]
]))

print(dpll([
    [('x1', True), ('x2', True)],
    [('x1', True), ('x2', False)],
    [('x1', False), ('x2', True)],
    [('x1', False), ('x2', False)]
]))

print(dpll([]))

print(dpll([[]]))

('satisfiable', {'x1': True, 'x2': True, 'x4': True, 'x3': False})
('unsatisfiable', {4: [3, 2], 5: [1, 4, 0, 4]})
('satisfiable', {})
('unsatisfiable', {1: [0]})
